# Network Operations

In this notebook we will assemble our project.  We will explore different functions on our dataset and compile them into a workflow ready for production.

The functions we will use will be a mix of `hub` based functions from our [MLRun Functions](http://github.com/mlrun/functions) repo, local and git based notebooks.

> The notebook should be run after generating the data in the [Generator Notebook](generator.ipynb)


we will start by setting up our environment, Loading MLRun and some utilities we will need

In [1]:
# Utils
import os
import json
import urllib
import numpy as np

# MLRun imports
from mlrun import mlconf

# Setup API Endpoint
mlconf.dbpath = 'http://mlrun-api:8080'

> If you are using another version of our `hub://` please set it up in the following cell.
* The url can parse {name} and {tag} to the given url

In [2]:
# Set Hub URL address if using a local version
# mlconf.hub_url = '/User/functions/{name}/function.yaml'

Now lets define our current project 

## Create a project from a git repository

In [3]:
from mlrun import new_project

# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = os.path.abspath('./')
remote_git = 'https://github.com/mlrun/demos.git'

# Create the project
newproj = new_project('network-operations', project_dir, init_git=False)

# We can update our project directory to the latest status by running
# newproj.pull()

Now that we have our project directory, lets forword our artifacts there to keep track of them

In [4]:
# Define an artifact path to keep track of where our artifacts are going
ARTIFACT_PATH =  os.path.join(os.path.abspath(newproj.context), 'artifacts')
mlconf.artifact_path = ARTIFACT_PATH

## Create and run functions

As we receive a new dataset, the first thing we would like to do is to explore it a bit, we can do that using our `describe` function in `mlrun/functions`

In [5]:
from mlrun import mount_v3io, new_model_server

In [6]:
# Import the functions
# Functions From hub
tag = 'dev-stream-concept-drift'
newproj.set_function(func=f'hub://aggregate:{tag}', name='aggregate')
newproj.set_function(func=f'hub://describe:{tag}', name='describe')
newproj.set_function(func=f'hub://feature_selection:{tag}', name="feature_selection")
newproj.set_function(func=f'hub://sklearn_classifier:{tag}', name='train')
newproj.set_function(func=f'hub://test_classifier:{tag}', name='test')
newproj.set_function(func=f'hub://model_server_tester:{tag}', name="model_server-tester")
newproj.set_function(func=f'hub://concept_drift:{tag}', name="concept_drift")
newproj.set_function(func=f'hub://stream_to_parquet:{tag}', name="s2p")
newproj.set_function(func=f'hub://virtual_drift:{tag}', name="virtual_drift")

# Streaming
src_path = os.path.abspath('notebooks/')
newproj.set_function(func=os.path.join(src_path, 'generator.ipynb'), name='generator')
newproj.set_function(func=os.path.join(src_path, 'preprocessor.ipynb'), name='create_feature_vector')
newproj.set_function(func=os.path.join(src_path, 'server.ipynb'), name="serving")
newproj.set_function(func=os.path.join(src_path, 'labeled_stream_creator.ipynb'), name="labeled_stream")

## Generate the dataset
If needed go to [Generator](./notebooks/generator.ipynb) and run the local workflow to generate the metrics dataset to `data/metrics`

## Run the functions locally to develop the workflow

now we can **Run** the function locally on our sample data, we would like to get some details on our `raw` data

## Register raw data as project level artifact

In [7]:
# Define base Dataset
import random
data_dir = os.path.join(os.path.abspath(newproj.context), 'data')
dataset_filename = random.choice(list(filter(lambda x: (x.endswith('pq') or x.endswith('parquet')), os.listdir(data_dir))))
metrics_path = os.path.join(data_dir, dataset_filename)
print(f'Selected {metrics_path} as base dataset, Prepearing dataset')

import pandas as pd
# Drop alternate error columns
label_column = 'is_error'
raw = pd.read_parquet(metrics_path)
raw = raw.drop([col for col in raw.columns if (col != label_column) & (col.endswith(label_column))], axis=1)
dataset_path = os.path.join(data_dir, 'metrics.pq')
raw.to_parquet(dataset_path)
print(f'Finished prepearing dataset {raw.shape}, logging artifact to store://{newproj.name}/metrics')

# Add to the project as a Dataset Artifact
from mlrun.artifacts import DatasetArtifact
from mlrun import get_or_create_ctx
mlctx = get_or_create_ctx('netops-project')
mlctx.log_dataset(key='metrics', df=raw, format='parquet', target_path=dataset_path)

Selected /User/mlrun-demos/demos/network-operations/data/20201223T221512-20201223T231512.parquet as base dataset, Prepearing dataset
Finished prepearing dataset (5768, 5), logging artifact to store://network-operations/metrics
> 2020-12-24 08:11:49,787 [info] logging run results to: http://mlrun-api:8080


<string>:6: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.


### Get statistics about the metrics data

In [8]:
from mlrun import NewTask
from mlrun.platforms import auto_mount

In [9]:
describe_task = NewTask(
    name="describe", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-metrics'},
    inputs={"table": metrics_path},
    artifact_path=ARTIFACT_PATH)

In [10]:
decsribe_run = newproj.func('describe').apply(auto_mount()).run(describe_task)

> 2020-12-20 15:15:49,311 [info] starting run describe uid=021253ee99794d1f884c4e658f38ed38 DB=http://mlrun-api:8080
> 2020-12-20 15:15:49,454 [info] Job is running in the background, pod: describe-bq95d
> 2020-12-20 15:15:59,545 [error] Failed to create pairplot histograms due to: Selected KDE bandwidth is 0. Cannot estiamte density.
> 2020-12-20 15:16:02,266 [info] run executed, status=completed
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...8f38ed38,0,Dec 20 15:15:56,completed,describe,v3io_user=orzkind=jobowner=orzhost=describe-bq95d,table,"key=summarylabel_column=is_errorclass_labels=['0', '1']plot_hist=Trueplot_dest=plots-metrics",,violinimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 021253ee99794d1f884c4e658f38ed38 --project network-operations , !mlrun logs 021253ee99794d1f884c4e658f38ed38 --project network-operations
> 2020-12-20 15:16:08,639 [info] run executed, status=completed


### Create the feature vector

We will use our [Aggregate](https://github.com/mlrun/functions/blob/master/aggregate/aggregate.ipynb) function to create rolling window features for our feature vector.

In doing so we hope that we could help our algorithms identify local errors by using a windowed trend

In [11]:
# Define aggregate task
from mlrun import NewTask
aggregate_task = NewTask(
    name='aggregate',
    params={'metrics': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
            'metric_aggs': ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
            'suffix': 'daily',
            'append_to_df': True,
            'window': 20,
            'center': False,
            'save_to': os.path.join('data', 'aggregate.pq'),
            'drop_na': True},
    inputs={'df_artifact': f'store://{newproj.name}/metrics'},
    handler='aggregate')

In [12]:
aggregate_run = newproj.func('aggregate').apply(mount_v3io()).run(aggregate_task)

> 2020-12-20 14:32:28,222 [info] starting run aggregate uid=6ff77910a9494210902c09baa558fdb9 DB=http://mlrun-api:8080
> 2020-12-20 14:32:28,342 [info] Job is running in the background, pod: aggregate-vj4w5
> 2020-12-20 14:32:31,813 [info] Aggregating /User/mlrun-demos/demos/network-operations/data/metrics.pq
> 2020-12-20 14:32:31,923 [info] Logging artifact
> 2020-12-20 14:32:32,190 [info] run executed, status=completed
<string>:6: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...a558fdb9,0,Dec 20 14:32:31,completed,aggregate,v3io_user=orzkind=jobowner=orzhost=aggregate-vj4w5,df_artifact,"metrics=['cpu_utilization', 'throughput', 'packet_loss', 'latency']metric_aggs=['mean', 'sum', 'std', 'var', 'min', 'max', 'median']suffix=dailyappend_to_df=Truewindow=20center=Falsesave_to=data/aggregate.pqdrop_na=True",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6ff77910a9494210902c09baa558fdb9 --project network-operations , !mlrun logs 6ff77910a9494210902c09baa558fdb9 --project network-operations
> 2020-12-20 14:32:34,459 [info] run executed, status=completed


### Get statistics about the feature vector

In [13]:
aggregate_describe_task = NewTask(
    name="describe-aggregate", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-aggregate',
            'sample': 0.3},
    inputs={"table": aggregate_run.outputs['aggregate']},
    artifact_path=ARTIFACT_PATH)

In [ ]:
aggregate_decsribe_run = newproj.func('describe').apply(mount_v3io()).run(aggregate_describe_task)

## Create workflow to train the model
After reviewing the data and creating the feature vector we move to training our model.  
For this task we will use an **LGBM** classifier.  To control the training process we will supply a `model_config` dictionary with the following parameters:
- **CLASS**: Model-specific parameters.
- **FIT**: Training parameters (like epoch when needed)
- **META**: Model and Package version

### Setup model configurations

In [24]:
model_configs = {
    "CLASS" : {
        "boosting_type"      : "gbdt",
        "num_leaves"         : 300,
        "max_depth"          : 50,
        "learning_rate"      : 0.1,
        "n_estimators"       : 300,
        "objective"          : "binary",
        "scale_pos_weight"   : 1,    
        "min_split_gain"     : 0.0,
        "min_child_samples"  : 20,
        "subsample"          : 1,
        "colsample_bytree"   : 1,
        "reg_alpha"          : 0,
        "reg_lambda"         : 1,
        "n_jobs"             : 16,
        "silent"             : True,
        "importance_type"    : "split",
        "random_state"       : 1},
    "FIT" : {
        "verbose"               : False
    },
    "META" : {
        "class" : "lightgbm.sklearn.LGBMClassifier",
        "version" : "2.3.1"
    }
}
model_config_path = os.path.join(os.path.abspath(newproj.context), 'data', 'lgb_model.json')
with open(model_config_path, 'w') as f:
          f.write(json.dumps(model_configs))

In [25]:
newproj.log_artifact('lgb_configs',
                     target_path = os.path.abspath(model_config_path))

In [ ]:
labeled_stream:
      name: labeled_stream
      kind: v3ioStream
      class: ""
      maxWorkers: 1
      url: "http://v3io-webapi:8081"
      attributes:
        jobBackoffLimit: 0
        port: 0
        consumerGroup: s2pt2
        numContainerWorkers: 0
        seekTo: latest
        readBatchSize: 64
        pollingIntervalMs: 500
        containerName: users
        streamPath: orz/mlrun-demos/demos/network-operations/streaming/labeled_stream
        sequenceNumberCommitInterval: 1s
        workerAllocationMode: pool
        sessionTimeout: 10s
        heartbeatInterval: 3s
        fetchDefault: 0
        intervalMs: 0
        maxBatchSize: 0
        maxBatchWaitMs: 0
        protocolVersion: 0
      disabled: false
      password: 036f0244-f7a4-453b-b7d9-786172282378

In [9]:
from nuclio.triggers import V3IOStreamTrigger, CronTrigger
import copy
fn = copy.copy(newproj.func('s2p'))
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
projdir = os.getcwd()
projdir_path = f"/{os.environ['V3IO_USERNAME']}{projdir[len('/User'):]}"
labeled_stream_path = os.path.join(projdir_path, 'streaming', 'labeled_stream')
print(projdir_path)
container = 'users'
fn.add_trigger('cron', CronTrigger(interval='1m'))
# fn.add_trigger('labeled_stream', V3IOStreamTrigger(url=f'https://webapi.default-tenant.app.lewpwntlsyrb.iguazio-cd1.com/{container}{labeled_stream_path}@s2p1'))
fn.add_trigger('labeled_stream', V3IOStreamTrigger(container=container,
                                                   path=labeled_stream_path,
                                                   seekTo='latest',
                                                   consumerGroup='s2p',
                                                   url="https://webapi.default-tenant.app.lewpwntlsyrb.iguazio-cd1.com"))
fn.set_envs({'window': 10,
             'features': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
             'save_to': os.path.join(projdir, 'streaming', 'inference_pq'),
             'base_dataset': '/User/mlrun-demos/demos/network-operations/artifacts/test_set_preds.parquet',
             'results_tsdb_container': 'users',
             'results_tsdb_table': 'orz/mlrun-demos/demos/network-operations/streaming/s2p_tsdb',
             'mount_path': '/users/orz',
             'mount_remote': '/User'})
print(fn.to_yaml())
fn.deploy()


/orz/mlrun-demos/demos/network-operations
kind: remote
metadata:
  name: stream-to-parquet
  tag: ''
  hash: 3595515d13a017cbac762871f2294a5f89f6ab40
  project: network-operations
  labels:
    author: orz
  categories:
  - ml
  - serve
spec:
  command: ''
  args: []
  image: ''
  entry_points:
    record_to_features:
      name: record_to_features
      doc: ''
      parameters:
      - name: record
        default: ''
      outputs:
      - default: ''
      lineno: 10
    init_context:
      name: init_context
      doc: ''
      parameters:
      - name: context
        default: ''
      outputs:
      - default: ''
      lineno: 17
    handler:
      name: handler
      doc: ''
      parameters:
      - name: context
        default: ''
      - name: event
        default: ''
      outputs:
      - default: ''
      lineno: 54
  description: Saves a stream to Parquet and can lunch drift detection task on it
  min_replicas: 1
  max_replicas: 1
  volumes: []
  volume_mounts: []
  en

'http://default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:32638'

In [37]:
from nuclio.triggers import V3IOStreamTrigger, CronTrigger
import copy
fn = copy.copy(newproj.func('s2p'))
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
projdir = os.getcwd()
projdir_path = projdir[len('/User'):]
container = 'users'
fn.add_trigger('cron', CronTrigger(interval='1m'))
fn.add_trigger('labeled_stream', V3IOStreamTrigger(url='https://webapi.default-tenant.app.lewpwntlsyrb.iguazio-cd1.com/users/orz/mlrun-demos/demos/network-operations/streaming/labeled_stream@s2p1'))
# fn.add_trigger('labeled_stream', V3IOStreamTrigger(container=container,
#                                                    path=os.path.join(projdir_path, 'streaming', 'labeled_stream'),
#                                                    seekTo='EARLIEST',
#                                                    partitions=0,
#                                                    consumerGroup='s2p'))
fn.set_envs({'window': 10,
             'features': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
             'save_to': os.path.join(projdir, 'streaming', 'inference_pq'),
             'base_dataset': '/User/mlrun-demos/demos/network-operations/artifacts/test_set_preds.parquet',
             'results_tsdb_container': 'users',
             'results_tsdb_table': 'orz/mlrun-demos/demos/network-operations/streaming/s2p_tsdb',
             'mount_path': '/users/orz',
             'mount_remote': '/User'})
print(fn.to_yaml())
fn.deploy()


kind: remote
metadata:
  name: stream-to-parquet
  tag: ''
  hash: 3595515d13a017cbac762871f2294a5f89f6ab40
  project: network-operations
  labels:
    author: orz
  categories:
  - ml
  - serve
spec:
  command: http://default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:32638
  args: []
  image: ''
  entry_points:
    record_to_features:
      name: record_to_features
      doc: ''
      parameters:
      - name: record
        default: ''
      outputs:
      - default: ''
      lineno: 10
    init_context:
      name: init_context
      doc: ''
      parameters:
      - name: context
        default: ''
      outputs:
      - default: ''
      lineno: 17
    handler:
      name: handler
      doc: ''
      parameters:
      - name: context
        default: ''
      - name: event
        default: ''
      outputs:
      - default: ''
      lineno: 54
  description: Saves a stream to Parquet and can lunch drift detection task on it
  min_replicas: 1
  max_replicas: 1
  volumes: []
  volume

'http://default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:32638'

### Create Pipeline Workflow

In [26]:
%%writefile src/workflow.py
from kfp import dsl
from mlrun import mount_v3io, mlconf
import os
from nuclio.triggers import V3IOStreamTrigger, CronTrigger

funcs = {}
projdir = os.getcwd()
projdir_path = f"/{os.environ['V3IO_USERNAME']}{projdir[len('/User'):]}"
labeled_stream_path = os.path.join(projdir_path, 'streaming', 'labeled_stream')
container = 'users'
full_path_projdir = os.path.join('/', container, os.environ["V3IO_USERNAME"], projdir[6:])

# Define a specific hub url?
# mlconf.hub_url = 'https://raw.githubusercontent.com/mlrun/functions/{tag}/{name}/function.yaml'
# mlconf.hub_url |= '/User/functions/{name}/function.yaml'

model_inference_stream = os.path.join(full_path_projdir, 'streaming', 'predictions')
labeled_stream = os.path.join(full_path_projdir, 'streaming', 'labeled_stream')

webapi_url = 'http://v3io-webapi:8081'
model_inference_url = f'{webapi_url}{model_inference_stream}'
labeled_stream_url = f'{webapi_url}{labeled_stream}'

def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        # Add V3IO Mount
        f.apply(mount_v3io())
        
        # Always pull images to keep updates
        f.spec.image_pull_policy = 'Always'
    
    # Define inference-stream related triggers
    functions['s2p'].add_trigger('labeled_stream', V3IOStreamTrigger(container=container,
                                                                     path=labeled_stream_path,
                                                                     seekTo='earliest',
                                                                     partitions=[0],
                                                                     consumerGroup='s2p',
                                                                     name='labeled_stream'))
    functions['generator'].add_trigger('cron', CronTrigger(interval='1m'))
    functions['labeled_stream'].add_trigger('cron', CronTrigger(interval='1m'))
    functions['create_feature_vector'].add_trigger('cron', CronTrigger(interval='1m'))
    functions['serving'].add_trigger('cron', CronTrigger(interval='1m'))
                
        
@dsl.pipeline(
    name='Network Operations Demo',
    description='Train a Failure Prediction LGBM Model over sensor data'
)
def kfpipeline(
        # aggregate
        df_artifact = os.path.join(projdir, 'data', 'metrics.pq'),
        metrics = ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
        metric_aggs = ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
        suffix = 'daily',
        window = 10,

        # describe
        describe_table = 'netops',
        describe_sample = 0.3,
        label_column = 'is_error',
        class_labels = [1, 0],
        plot_hist = True,
    
        # Feature selection
        k = 5,
        min_votes = 3,
    
        # Train
        sample_size      = -1,        # -n for random sample of n obs, -1 for entire dataset, +n for n consecutive rows
        test_size        = 0.1,       # 10% set aside
        train_val_split  = 0.75,      # remainder split into train and val
    
        # Test
        predictions_col = 'predictions',
    
        # Deploy
        deploy_streaming = True,
        aggregate_fn_url = 'hub://aggregate',
        streaming_features_table = os.path.join(projdir, 'streaming', 'features'),
        streaming_predictions_table = os.path.join(projdir, 'streaming', 'predictions'),
    
        # Streaming
        streaming_metrics_table = os.path.join(projdir, 'streaming', 'metrics'),
        generator_metrics_configuration = os.path.join(projdir, 'src', 'metric_configurations.yaml'),
    
        # labeled stream creator
        streaming_labeled_table = labeled_stream,
        
        # Concept drift
        deploy_concept_drift = True,
        secs_to_generate = 10,
        concept_drift_models = ['ddm', 'eddm', 'pagehinkley'],
        output_tsdb = os.path.join(projdir, 'streaming', 'drift_tsdb'),
        input_stream = labeled_stream_url,
        output_stream = os.path.join(projdir, 'streaming', 'drift_stream'),
        streaming_parquet_table =  os.path.join(projdir, 'streaming', 'inference_pq'),
    
        # Virtual drift
        results_tsdb_container = 'users',
        results_tsdb_table = os.path.join(full_path_projdir[7:], 'streaming', 'drift_magnitude')
    ):
    
    # Run preprocessing on the data
    aggregate = funcs['aggregate'].as_step(name='aggregate',
                                                  params={'metrics': metrics,
                                                          'metric_aggs': metric_aggs,
                                                          'suffix': suffix,
                                                          'window': window},
                                                  inputs={'df_artifact': df_artifact},
                                                  outputs=['aggregate'],
                                                  handler='aggregate',
                                                  image='mlrun/ml-models')

    describe = funcs['describe'].as_step(name='describe-aggregation',
                                        handler="summarize",  
                                        params={"key": f"{describe_table}_aggregate", 
                                                "label_column": label_column, 
                                                'class_labels': class_labels,
                                                'plot_hist': plot_hist,
                                                'plot_dest': 'plots/aggregation',
                                                'sample': describe_sample},
                                        inputs={"table": aggregate.outputs['aggregate']},
                                        outputs=["summary", "scale_pos_weight"])
    
    feature_selection = funcs['feature_selection'].as_step(name='feature_selection',
                                                           handler='feature_selection',
                                                           params={'k': k,
                                                                   'min_votes': min_votes,
                                                                   'label_column': label_column},
                                                           inputs={'df_artifact': aggregate.outputs['aggregate']},
                                                           outputs=['feature_scores', 
                                                                    'max_scaled_scores_feature_scores'
                                                                    'selected_features_count', 
                                                                    'selected_features'],
                                                           image='mlrun/ml-models')
    
    describe = funcs['describe'].as_step(name='describe-feature-vector',
                                            handler="summarize",  
                                            params={"key": f'{describe_table}_features', 
                                                    "label_column": label_column, 
                                                    'class_labels': class_labels,
                                                    'plot_hist': plot_hist,
                                                    'plot_dest': 'plots/feature_vector'},
                                            inputs={"table": feature_selection.outputs['selected_features']},
                                            outputs=["summary", "scale_pos_weight"])
    
    train = funcs['train'].as_step(name='train',
                                   params={"sample"          : sample_size, 
                                           "label_column"    : label_column,
                                           "test_size"       : test_size,
                                           "train_val_split" : train_val_split},
                                   inputs={"dataset"         : feature_selection.outputs['selected_features']},
                                   hyperparams={'model_pkg_class': ["sklearn.ensemble.RandomForestClassifier", 
                                                                    "sklearn.linear_model.LogisticRegression",
                                                                    "sklearn.ensemble.AdaBoostClassifier"]},
                                   selector='max.accuracy',
                                   outputs=['model', 'test_set'],
                                   image='mlrun/ml-models')
    
    test = funcs['test'].as_step(name='test',
                                 handler='test_classifier',
                                 params={'label_column': label_column,
                                         'predictions_column': predictions_col},
                                 inputs={'models_path': train.outputs['model'],
                                         'test_set': train.outputs['test_set']},
                                 outputs=['test_set_preds'],
                                 image='mlrun/ml-models')

    
    with dsl.Condition(deploy_streaming == True):
        
        # deploy the model using nuclio functions
        deploy = funcs['serving'].deploy_step(env={'model_path': train.outputs['model'],
                                                   'FEATURES_TABLE': streaming_features_table,
                                                   'PREDICTIONS_TABLE': streaming_predictions_table,
                                                   'prediction_col': predictions_col}, 
                                              tag='v1')

        # test out new model server (via REST API calls)
        tester = funcs["model_server-tester"].as_step(name='model-tester',
                                                      params={'addr': deploy.outputs['endpoint'], 
                                                              'model': "predictor",
                                                              'label_column': label_column},
                                                      inputs={'table': train.outputs['test_set']},
                                                      outputs=['test_set_preds'])
    
        # Streaming demo functions
        preprocessor = funcs['create_feature_vector'].deploy_step(env={ 'aggregate_fn_url': aggregate_fn_url,
                                                                'METRICS_TABLE': streaming_metrics_table,
                                                                'FEATURES_TABLE': streaming_features_table,
                                                                'metrics': metrics,
                                                                'metric_aggs': metric_aggs,
                                                                'suffix': suffix,
                                                                'base_dataset': train.outputs['test_set'],
                                                                'label_col': label_column}).after(tester)

        labeled_stream_creator = funcs['labeled_stream'].deploy_step(env={'METRICS_TABLE': streaming_metrics_table,
                                                                                  'PREDICTIONS_TABLE': streaming_predictions_table,
                                                                                  'OUTPUT_STREAM': streaming_labeled_table,
                                                                                  'label_col': label_column,
                                                                                  'prediction_col': predictions_col}).after(tester)

        generator = funcs['generator'].deploy_step(env={'SAVE_TO': streaming_metrics_table,
                                                        'SECS_TO_GENERATE': secs_to_generate,
                                                        'METRICS_CONFIGURATION_FILEPATH': generator_metrics_configuration}).after(preprocessor)
        
        with dsl.Condition(deploy_concept_drift == True):

            concept_builder = funcs['concept_drift'].deploy_step(skip_deployed=True)

            concept_drift = funcs['concept_drift'].as_step(name='concept_drift_deployer',
                                                           params={'models': concept_drift_models,
                                                                   'label_col': label_column,
                                                                   'prediction_col': predictions_col,
                                                                   'output_tsdb': output_tsdb,
                                                                   'input_stream': f'{input_stream}@cds',
                                                                   'output_stream': output_stream},
                                                           inputs={'base_dataset': test.outputs['test_set_preds']},
                                                           artifact_path=mlconf.artifact_path,
                                                           image=concept_builder.outputs['image']).after(labeled_stream_creator)

            s2p = funcs['s2p'].deploy_step(env={'window': 10,
                                                'features': metrics,
                                                'save_to': streaming_parquet_table,
                                                'base_dataset': test.outputs['test_set_preds'],
                                                'results_tsdb_container': 'users',
                                                'results_tsdb_table': results_tsdb_table,
                                                'mount_path': '/users/orz',
                                                'mount_remote': '/User'}).after(labeled_stream_creator)
    

Overwriting src/workflow.py


## Add workflow

In [28]:
newproj.set_workflow('main', os.path.join(os.path.abspath(newproj.context), 'src', 'workflow.py'))

## Save Project

In [29]:
newproj.save(os.path.join(newproj.context, 'project.yaml'))

### Run workflow

In [30]:
newproj.run('main', artifact_path=ARTIFACT_PATH, dirty=True)

> 2020-12-23 14:16:49,325 [info] using in-cluster config.


/conda/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['cpu_utilization', 'throughput', 'packet_loss', 'latency']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['mean', 'sum', 'std', 'var', 'min', 'max', 'median']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "10".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Mi

> 2020-12-23 14:16:50,708 [info] Pipeline run id=96bdd114-87a1-4ec3-802f-02f7232b2623, check UI or DB for progress


'96bdd114-87a1-4ec3-802f-02f7232b2623'